In [10]:
from brfinance import CVMAsyncBackend
import pandas as pd, numpy as np
from datetime import date
import statistics
import matplotlib.pyplot as plt

In [11]:
cvm_httpclient = CVMAsyncBackend()

# Dict de códigos CVM para todas as empresas
cvm_codes = cvm_httpclient.get_cvm_codes()

# Dict de todas as categorias de busca disponíveis (Fato relevante, DFP, ITR, etc.)
categories = cvm_httpclient.get_consulta_externa_cvm_categories()

# Realizando busca por Empresa
ano = 2017 #int(input("Qual o ano inicial para a pesquisa? "))

start_date = date(ano, 12, 31)
end_dt = date.today()
cvm_codes_list = ['21067'] # Moura_Dubeux 3694
category = ["EST_4"] # Códigos de categoria para DFP, ITR e fatos relevantes #, "EST_3", "IPE_4_-1_-1"
last_ref_date = False # Se "True" retorna apenas o último report no intervalo de datas

# Busca
search_result = cvm_httpclient.get_consulta_externa_cvm_results(
    cod_cvm=cvm_codes_list,
    start_date=start_date,
    end_date=end_dt,
    last_ref_date=last_ref_date,
    category=category
    )

# Filtrar dataframe de busca para DFP e Status Ativo
search_result = search_result[
    (search_result['categoria']=="DFP - Demonstrações Financeiras Padronizadas")]
search_result = search_result[search_result['status']=="Ativo"]
search_result = search_result[search_result['ref_date']>=str(start_date)]
search_result = search_result[pd.to_numeric(search_result['numero_seq_documento'], errors='coerce').notnull()]

reports_list = [
    'Balanço Patrimonial Ativo',
    'Balanço Patrimonial Passivo',
    'Demonstração do Resultado',
    'Demonstração do Fluxo de Caixa'] # Se None retorna todos os demonstrativos disponíveis.

#Lista vazia para armazenas os dataframes
dfs = []

# Obter demonstrativos
for index, row in search_result.iterrows():
    empresa = f"{row['cod_cvm']} - {cvm_codes[row['cod_cvm']]}"

    reports = cvm_httpclient.get_report(row["numero_seq_documento"], row["codigo_tipo_instituicao"], reports_list=reports_list)
    for report in reports:
        reports[report]["cod_cvm"] = row["cod_cvm"]

        #Adicionar os dataframes em uma lista
        dfs.append(reports[report])    

#Concatenar os dataframes em um único dataframe
df_concat = pd.concat(dfs,ignore_index=True)


# Salvar o dataframe no arquivo excel, repetindo as linhas das DFPs conforme os anos do filtro
df_concat.to_excel('Dados_CVM.xlsx')

# Criar pivot table
pivot_table = pd.pivot_table(df_concat, index=["Conta", "Descrição"], values=["Valor"], aggfunc=lambda x: "\n".join(str(i) for i in x))

# Salvar pivot table em um arquivo Excel
pivot_table.to_excel('Dados_CVM_Filtrados.xlsx')

# Carregar o arquivo pivot_table.xlsx em um DataFrame pandas
df_pivot = pd.read_excel('Dados_CVM_Filtrados.xlsx')

# Obter o número máximo de valores em uma única célula "Valor" em todo o DataFrame
max_values = int(df_pivot['Valor'].str.count('\n').max()) + 1
value_cols = [f'{ano+i}' for i in reversed(range(max_values))]
df_pivot[value_cols] = df_pivot['Valor'].str.split('\n', expand=True)

# Separar os valores da coluna "Valor" em colunas separadas
df_pivot[value_cols] = df_pivot['Valor'].str.split('\n', expand=True)

# Remover a coluna "Valor" original
df_pivot = df_pivot.drop(columns=['Valor'])

# Ordenar colunas em ordem crescente, mantendo as duas primeiras colunas
all_cols = list(df_pivot.columns)
ordered_cols = all_cols[:2] + sorted(all_cols[2:])

# Reagrupamento das colunas no DataFrame
df_pivot = df_pivot.reindex(columns=ordered_cols)

#Filtrar dados
df_pivot.query('Conta == "3.01" | Conta == "3.02" | Conta == "6.01.01.03" \
    | Conta == "3.06" | Conta == "3.08" | Conta == "3.04" | Conta == "3.11" \
    | Conta == "6.01.02" | Conta == "6.02.04"', inplace = True)

#Formatar estilo
#df_pivot.style.format(decimal=",")

# Salvar o DataFrame atualizado em um arquivo Excel
df_pivot.to_excel("Case_Base_Futuro.xlsx", index=False)


c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\brfinance\responses.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  response_df = response_df.append(search_results_df)
c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\brfinance\responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\brfinance\responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-pac

In [12]:
def calculate_rate(array):
    n = len(array) - 1
    initial_value = array[0]
    final_value = array[-1]
    rate = (final_value / initial_value) ** (1 / n) - 1
    return rate

In [13]:
############################################ Razões Futuras ###############################################3##
df_futuro = pd.read_excel('Case_Base_Futuro.xlsx')

yoy_growth_rate = round(calculate_rate((df_futuro.iloc[0][2:]).astype(float)),3)#Taxa de crescimento anual das vendas

cb_302 = sum(df_futuro.iloc[1][2:]/df_futuro.iloc[0][2:])/(len(df_futuro.columns) - 2)#Custo dos Bens e/ou Serviços Vendidos
cb_304 = sum(df_futuro.iloc[2][2:]/df_futuro.iloc[0][2:])/(len(df_futuro.columns) - 2)#Despesas/Receitas Operacionais
cb_306 = sum(abs(df_futuro.iloc[3][2:]/df_futuro.iloc[0][2:]))/(len(df_futuro.columns) - 2)#Resultado Financeiro
cb_308 = sum(df_futuro.iloc[4][2:]/df_futuro.iloc[0][2:])/(len(df_futuro.columns) - 2)#Imposto de Renda e Contribuição Social sobre o Lucro
cb_6101 = sum(df_futuro.iloc[6][2:]/df_futuro.iloc[0][2:])/(len(df_futuro.columns) - 2)#Depreciação e Amortização
cb_6102 = sum(df_futuro.iloc[6][2:]/df_futuro.iloc[0][2:])/(len(df_futuro.columns) - 2)#Variações nos Ativos e Passivos
cb_6204 = sum(abs(df_futuro.iloc[8][2:]/df_futuro.iloc[0][2:]))/(len(df_futuro.columns) - 2)#Adições ao Imobilizado


In [14]:
############################################ Anos Futuros ###############################################################
#df_futuro = pd.read_excel('Case_Base_Futuro.xlsx')

#Retirando o index, são 16 linhas

#Criando novas colunas 
for i in range(5):
    #data_row = np.zeros(16)
    data_row = [(df_futuro.iloc[0][len(df_futuro.columns)-1]*(1 + yoy_growth_rate)),#Vendas\
        (df_futuro.iloc[0][len(df_futuro.columns)-1]*(1 + yoy_growth_rate)) * (cb_302),#Custo dos Bens e/ou Serviços Vendidos\
        (df_futuro.iloc[0][len(df_futuro.columns)-1]*(1 + yoy_growth_rate)) * (cb_304),#Despesas/Receitas Operacionais\
        (df_futuro.iloc[0][len(df_futuro.columns)-1]*(1 + yoy_growth_rate)) * (cb_306),#Resultado Financeiro\
        (df_futuro.iloc[0][len(df_futuro.columns)-1]*(1 + yoy_growth_rate)) * (cb_308),#Imposto de Renda e Contribuição Social sobre o Lucro\
        (df_futuro.iloc[0][len(df_futuro.columns)-1]*(1 + yoy_growth_rate)) + (df_futuro.iloc[0][len(df_futuro.columns)-1] * (cb_302)) + (df_futuro.iloc[0][len(df_futuro.columns)-1] * (cb_304)) + (df_futuro.iloc[0][len(df_futuro.columns)-1] * (cb_306)) + (df_futuro.iloc[0][len(df_futuro.columns)-1] * (cb_308)),\
        df_futuro.iloc[6][len(df_futuro.columns)-1] * (1 + cb_6101),#Depreciação e Amortização\
        (df_futuro.iloc[7][len(df_futuro.columns)-1]) * (1 + (1 + yoy_growth_rate)),#Variações nos Ativos e Passivos\
        df_futuro.iloc[8][len(df_futuro.columns)-1] * (1 + (1 + yoy_growth_rate))]#Adições ao Imobilizado
    df_futuro[f'{ano + len(df_futuro.columns)-2}'] = data_row

df_futuro.to_excel("Case_Base_Futuro.xlsx", index=False)

In [15]:
############################################# Lucro Bruto #########################################################

# Ler o arquivo "Case_Base_Futuro.xlsx" em um DataFrame
df_futuro = pd.read_excel("Case_Base_Futuro.xlsx")

#Inicializando 
Lucro_Bruto = []
for i in range((len(df_futuro.columns)-2)):
    Lucro_Bruto.append(pd.to_numeric(df_futuro.loc[df_futuro["Conta"].isin(["3.01", "3.02"]), f"{ano+i}"]).sum())

# Criar novo DataFrame com a nova linha
new_row = pd.DataFrame([["3.01+3.02", "Lucro Bruto"] + Lucro_Bruto], columns=df_futuro.columns)

df_futuro = pd.concat([df_futuro, new_row], ignore_index=True)


In [16]:
############################################# EBIT #########################################################

# selecionando a linha Depreciação e Armotização e multiplicando os valores por -1
#for i in range((len(df.columns)-2)):
    #df.loc[6,f"{ano+i}"] = df.loc[6,f"{ano+i}"] * -1

#Inicializando 
EBIT = []
for i in range((len(df_futuro.columns)-2)):
    df_futuro.loc[6,f"{ano+i}"] = df_futuro.loc[6,f"{ano+i}"] * -1 #selecionando a linha Depreciação e Armotização e multiplicando os valores por -1
    EBIT.append(pd.to_numeric(df_futuro.loc[df_futuro["Conta"].isin(["3.01+3.02", "3.04", "6.01.01.03"]), f"{ano+i}"]).sum())

# Criar novo DataFrame com a nova linha
new_row = pd.DataFrame([["fx1", "EBIT"] + EBIT], columns=df_futuro.columns)

# Concatenar o novo DataFrame com o DataFrame original
df_futuro = pd.concat([df_futuro, new_row], ignore_index=True)


# Salvar o DataFrame atualizado em um arquivo Excel
#df_futuro.to_excel("Case_Base_Futuro.xlsx", index=False)

In [17]:
############################################# NOPAT #########################################################
#Inicializando 
NOPAT = []
for i in range((len(df_futuro.columns)-2)):
    NOPAT.append(pd.to_numeric(df_futuro.loc[10,f"{ano+i}"] * (1 - abs(df_futuro.loc[4,f"{ano+i}"]/df_futuro.loc[10,f"{ano+i}"]))))

# Criar novo DataFrame com a nova linha
new_row = pd.DataFrame([["fx2", "NOPAT"] + NOPAT], columns=df_futuro.columns)

# Concatenar o novo DataFrame com o DataFrame original
df_futuro = pd.concat([df_futuro, new_row], ignore_index=True)

# Salvar o DataFrame atualizado em um arquivo Excel
#df.to_excel("Case_Base_Futuro.xlsx", index=False)

In [18]:
############################################# FreeCashFlowtoFirm #########################################################

#Inicializando 
FreeCash = []
for i in range((len(df_futuro.columns)-2)):
    FreeCash.append(pd.to_numeric(df_futuro.loc[11,f"{ano+i}"] + df_futuro.loc[6,f"{ano+i}"] - df_futuro.loc[7,f"{ano+i}"] - df_futuro.loc[8,f"{ano+i}"]))

# Criar novo DataFrame com a nova linha
new_row = pd.DataFrame([["fx3", "FreeCashFlow"] + FreeCash], columns=df_futuro.columns)

# Concatenar o novo DataFrame com o DataFrame original
df_futuro = pd.concat([df_futuro, new_row], ignore_index=True)

# Salvar o DataFrame atualizado em um arquivo Excel
df_futuro.to_excel("Case_Base_Futuro.xlsx", index=False)

In [19]:
fcf = [] #Free Cash Flow
for i in range(len(df_futuro.columns)-2):
    fcf.append(df_futuro[f'{ano + i}'].iloc[12])

#print(type(fcf))

In [20]:
wacc = 0.20
perp_gwth_rate = 0.015
dfc_factor = [1/(1 + wacc) ** x for x in range(1, len(df_futuro.columns)-1)]
term_value = (df_futuro[f'{ano}'].iloc[12] * (1 + perp_gwth_rate) / (wacc - perp_gwth_rate)) * dfc_factor[-1]

In [22]:
print(term_value)

42263.19196959832


In [21]:
df_futuro = pd.read_excel('Case_Base_Futuro.xlsx')
df_futuro = df_futuro.T

npv = df_futuro[12][2:] * dfc_factor
comp_value = npv.sum() + term_value
comp_value

707033.7225039889